#Install Library

In [ ]:
!pip install pymupdf langchain langchain-community chromadb sentence-transformers langchain_huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 114.6 MB/s eta 

In [ ]:
!pip install rank_bm25

In [ ]:
!pip install torch

#Inizialization

In [ ]:
import os
import fitz  # PyMuPDF
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PyMuPDFLoader
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from transformers import AutoModel, AutoTokenizer, AutoModelForSeq2SeqLM
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import ndcg_score
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
import time
import numpy as np
import torch
import torch.nn.functional as F

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#To make a vectordb/Pre-processing


In [ ]:
# Define the parent directory containing subfolders with PDFs
parent_dir = "/content/drive/MyDrive/Skripsi_C14210098/Dataset RAG/Introduction to AI"

# List all PDF file paths recursively
pdf_files = []
for root, _, files in os.walk(parent_dir):
    for file in files:
        if file.endswith(".pdf"):
            pdf_files.append(os.path.join(root, file))

# Load each PDF file as a single document
intro_to_AI_docs = []
for pdf_path in pdf_files:
    loader = PyMuPDFLoader(pdf_path)
    pages = loader.load()  # Load all pages

    # Merge all pages into a single document
    merged_text = "\n".join([page.page_content for page in pages])

    # Store as one document per file
    intro_to_AI_docs.append(Document(
        page_content=merged_text,
        metadata={"source": pdf_path, "module": "Introduction to AI"}
    ))

# Check the number of PDFs loaded (should match actual file count)
print(f"Total PDFs loaded: {len(intro_to_AI_docs)}")

Total PDFs loaded: 78


In [ ]:
# Define the parent directory containing subfolders with PDFs
parent_dir = "/content/drive/MyDrive/Skripsi_C14210098/Dataset RAG/Heuristic Search"

# List all PDF file paths recursively
pdf_files = []
for root, _, files in os.walk(parent_dir):
    for file in files:
        if file.endswith(".pdf"):
            pdf_files.append(os.path.join(root, file))

# Load each PDF file as a single document
heuristic_docs = []
for pdf_path in pdf_files:
    loader = PyMuPDFLoader(pdf_path)
    pages = loader.load()  # Load all pages

    # Merge all pages into a single document
    merged_text = "\n".join([page.page_content for page in pages])

    # Store as one document per file
    heuristic_docs.append(Document(
        page_content=merged_text,
        metadata={"source": pdf_path, "module": "Heuristic Search"}
    ))

# Check the number of PDFs loaded (should match actual file count)
print(f"Total PDFs loaded: {len(heuristic_docs)}")

Total PDFs loaded: 163


In [ ]:
# Define the parent directory containing subfolders with PDFs
parent_dir = "/content/drive/MyDrive/Skripsi_C14210098/Dataset RAG/Rule based AI and Fuzzy Logic"

# List all PDF file paths recursively
pdf_files = []
for root, _, files in os.walk(parent_dir):
    for file in files:
        if file.endswith(".pdf"):
            pdf_files.append(os.path.join(root, file))

# Load each PDF file as a single document
rule_base_fuzzy_docs = []
for pdf_path in pdf_files:
    loader = PyMuPDFLoader(pdf_path)
    pages = loader.load()  # Load all pages

    # Merge all pages into a single document
    merged_text = "\n".join([page.page_content for page in pages])

    # Store as one document per file
    rule_base_fuzzy_docs.append(Document(
        page_content=merged_text,
        metadata={"source": pdf_path, "module": "Rule based AI and Fuzzy Logic"}
    ))

# Check the number of PDFs loaded (should match actual file count)
print(f"Total PDFs loaded: {len(rule_base_fuzzy_docs)}")

Total PDFs loaded: 125


In [ ]:
# Define the parent directory containing subfolders with PDFs
parent_dir = "/content/drive/MyDrive/Skripsi_C14210098/Dataset RAG/Genetic Algorithm and Nature Inspired Algorithms"

# List all PDF file paths recursively
pdf_files = []
for root, _, files in os.walk(parent_dir):
    for file in files:
        if file.endswith(".pdf"):
            pdf_files.append(os.path.join(root, file))

# Load each PDF file as a single document
GA_NIA_docs = []
for pdf_path in pdf_files:
    loader = PyMuPDFLoader(pdf_path)
    pages = loader.load()  # Load all pages

    # Merge all pages into a single document
    merged_text = "\n".join([page.page_content for page in pages])

    # Store as one document per file
    GA_NIA_docs.append(Document(
        page_content=merged_text,
        metadata={"source": pdf_path, "module": "Genetic Algorithm and Nature Inspired Algorithms"}
    ))

# Check the number of PDFs loaded (should match actual file count)
print(f"Total PDFs loaded: {len(GA_NIA_docs)}")

Total PDFs loaded: 102


In [ ]:
# Define the parent directory containing subfolders with PDFs
parent_dir = "/content/drive/MyDrive/Skripsi_C14210098/Dataset RAG/Machine Learning"

# List all PDF file paths recursively
pdf_files = []
for root, _, files in os.walk(parent_dir):
    for file in files:
        if file.endswith(".pdf"):
            pdf_files.append(os.path.join(root, file))

# Load each PDF file as a single document
ML_docs = []
for pdf_path in pdf_files:
    loader = PyMuPDFLoader(pdf_path)
    pages = loader.load()  # Load all pages

    # Merge all pages into a single document
    merged_text = "\n".join([page.page_content for page in pages])

    ML_docs.append(Document(
        page_content=merged_text,
        metadata={"source": pdf_path, "module": "Machine Learning"}
    ))

# Check the number of PDFs loaded (should match actual file count)
print(f"Total PDFs loaded: {len(ML_docs)}")

MuPDF error: syntax error: cannot find XObject resource 'arial-minus'

MuPDF error: syntax error: cannot find XObject resource 'arial-minus'

MuPDF error: syntax error: cannot find XObject resource 'arial-minus'

MuPDF error: syntax error: cannot find XObject resource 'arial-minus'

MuPDF error: syntax error: cannot find XObject resource 'arial-minus'

MuPDF error: syntax error: could not parse color space (1778 0 R)

MuPDF error: syntax error: could not parse color space (1778 0 R)

MuPDF error: syntax error: could not parse color space (243 0 R)

Total PDFs loaded: 365


In [ ]:
# Define the parent directory containing subfolders with PDFs
parent_dir = "/content/drive/MyDrive/Skripsi_C14210098/Dataset RAG/Ensemble of Machine Learning"

# List all PDF file paths recursively
pdf_files = []
for root, _, files in os.walk(parent_dir):
    for file in files:
        if file.endswith(".pdf"):
            pdf_files.append(os.path.join(root, file))

# Load each PDF file as a single document
ensembel_ML_docs = []
for pdf_path in pdf_files:
    loader = PyMuPDFLoader(pdf_path)
    pages = loader.load()  # Load all pages

    # Merge all pages into a single document
    merged_text = "\n".join([page.page_content for page in pages])

    # Store as one document per file
    ensembel_ML_docs.append(Document(
        page_content=merged_text,
        metadata={"source": pdf_path, "module": "Ensemble of Machine Learning"}
    ))

# Check the number of PDFs loaded (should match actual file count)
print(f"Total PDFs loaded: {len(ensembel_ML_docs)}")

Total PDFs loaded: 163


In [ ]:
# Define the parent directory containing subfolders with PDFs
parent_dir = "/content/drive/MyDrive/Skripsi_C14210098/Dataset RAG/Neural Network, Deep Learning, and Generative AI"

# List all PDF file paths recursively
pdf_files = []
for root, _, files in os.walk(parent_dir):
    for file in files:
        if file.endswith(".pdf"):
            pdf_files.append(os.path.join(root, file))

# Load each PDF file as a single document
NN_DL_Gen_AI_docs = []
for pdf_path in pdf_files:
    loader = PyMuPDFLoader(pdf_path)
    pages = loader.load()  # Load all pages

    # Merge all pages into a single document
    merged_text = "\n".join([page.page_content for page in pages])

    # Store as one document per file
    NN_DL_Gen_AI_docs.append(Document(
        page_content=merged_text,
        metadata={"source": pdf_path, "module": "Neural Network, Deep Learning, and Generative AI"}
    ))

# Check the number of PDFs loaded (should match actual file count)
print(f"Total PDFs loaded: {len(NN_DL_Gen_AI_docs)}")

MuPDF error: syntax error: could not parse color space (908 0 R)

MuPDF error: syntax error: cannot find ExtGState resource 'gs0'

MuPDF error: syntax error: cannot find ExtGState resource 'gs0'

MuPDF error: syntax error: cannot find ExtGState resource 'gs0'

MuPDF error: syntax error: cannot find ExtGState resource 'gs0'

MuPDF error: syntax error: cannot find ExtGState resource 'gs0'

Total PDFs loaded: 279


In [ ]:
books=[]

In [ ]:
loader = PyMuPDFLoader('/content/drive/MyDrive/Skripsi_C14210098/Dataset RAG/Buku materi pembelajaran/Artificial-Intelligence-A-Modern-Approach-4th-Edition-1-compressed.pdf')
pages = loader.load()  # Load all pages

# Merge all pages into a single document
merged_text = "\n".join([page.page_content for page in pages])
books.append(Document(
        page_content=merged_text,
        metadata={"source": pdf_path, "module": "Neural Network, Deep Learning, and Generative AI, Machine Learning, Ensemble of Machine Learning, Introduction to AI, Heuristic Search, Rule based AI and Fuzzy Logic, Genetic Algorithm and Nature Inspired Algorithms"}
    ))

In [ ]:
loader = PyMuPDFLoader('/content/drive/MyDrive/Skripsi_C14210098/Dataset RAG/Buku materi pembelajaran/Hands_On_Machine_Learning_with_Scikit_Learn_and_TensorFlow.pdf')
pages = loader.load()  # Load all pages

# Merge all pages into a single document
merged_text = "\n".join([page.page_content for page in pages])
books.append(Document(
        page_content=merged_text,
        metadata={"source": pdf_path, "module": "Neural Network, Deep Learning, and Generative AI, Machine Learning, Ensemble of Machine Learning"}
    ))

In [ ]:
loader = PyMuPDFLoader('/content/drive/MyDrive/Skripsi_C14210098/Dataset RAG/Buku materi pembelajaran/Genetic Algorithms with Python by Eyal Wirsansky .pdf')
pages = loader.load()  # Load all pages

# Merge all pages into a single document
merged_text = "\n".join([page.page_content for page in pages])
books.append(Document(
        page_content=merged_text,
        metadata={"source": pdf_path, "module": "Genetic Algorithm and Nature Inspired Algorithms"}
    ))

In [ ]:
loader = PyMuPDFLoader('/content/drive/MyDrive/Skripsi_C14210098/Dataset RAG/Buku materi pembelajaran/a-machine-learning-artificial-intelligence-approach-to-institutional-effectiveness-in-higher-education-1nbsped-9781789738995-9781789739008.pdf')
pages = loader.load()  # Load all pages

# Merge all pages into a single document
merged_text = "\n".join([page.page_content for page in pages])
books.append(Document(
        page_content=merged_text,
        metadata={"source": pdf_path, "module": "Neural Network, Deep Learning, and Generative AI, Machine Learning"}
    ))

In [ ]:
print(f"Total PDFs loaded: {len(books)}")

Total PDFs loaded: 4


In [ ]:
# Remove '\n' from page_content
for doc in intro_to_AI_docs:
    doc.page_content = doc.page_content.replace('\n', ' ')
for doc in heuristic_docs:
    doc.page_content = doc.page_content.replace('\n', ' ')
for doc in rule_base_fuzzy_docs:
    doc.page_content = doc.page_content.replace('\n', ' ')
for doc in GA_NIA_docs:
    doc.page_content = doc.page_content.replace('\n', ' ')
for doc in ML_docs:
    doc.page_content = doc.page_content.replace('\n', ' ')
for doc in ensembel_ML_docs:
    doc.page_content = doc.page_content.replace('\n', ' ')
for doc in NN_DL_Gen_AI_docs:
    doc.page_content = doc.page_content.replace('\n', ' ')
for doc in books:
    doc.page_content = doc.page_content.replace('\n', ' ')

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=50)

In [ ]:
intro_to_AI_chunks = text_splitter.split_documents(intro_to_AI_docs)
heuristic_chunks = text_splitter.split_documents(heuristic_docs)
rule_base_fuzzy_chunks = text_splitter.split_documents(rule_base_fuzzy_docs)
GA_NIA_chunks = text_splitter.split_documents(GA_NIA_docs)
ML_chunks = text_splitter.split_documents(ML_docs)
ensembel_ML_chunks = text_splitter.split_documents(ensembel_ML_docs)
NN_DL_Gen_AI_chunks = text_splitter.split_documents(NN_DL_Gen_AI_docs)
books_chunks = text_splitter.split_documents(books)

In [ ]:
all_chunks = intro_to_AI_chunks + heuristic_chunks + rule_base_fuzzy_chunks + GA_NIA_chunks + ML_chunks + ensembel_ML_chunks + NN_DL_Gen_AI_chunks+books_chunks

In [ ]:
len(all_chunks)

12547

In [ ]:
persist_directory = '/content/drive/MyDrive/Skripsi_C14210098/5000_DB'

model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#To make vectordb
vectordb = Chroma.from_documents(documents=all_chunks,
                                 embedding=hf,
                                 persist_directory=persist_directory)

In [ ]:
len(vectordb)

12547

#Pengujian dataset

##Load Vector Database

In [ ]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
persist_directory = '/content/drive/MyDrive/Skripsi_C14210098/DB_2000'
vectordb_2000 = Chroma(embedding_function=hf, persist_directory=persist_directory)
vectordb_2000.persist()

<ipython-input-7-2212947135>:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb_2000 = Chroma(embedding_function=hf, persist_directory=persist_directory)
<ipython-input-7-2212947135>:3: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb_2000.persist()


In [ ]:
persist_directory = '/content/drive/MyDrive/Skripsi_C14210098/DB_3000'
vectordb_3000 = Chroma(embedding_function=hf, persist_directory=persist_directory)
vectordb_3000.persist()

In [ ]:
persist_directory = '/content/drive/MyDrive/Skripsi_C14210098/DB_4000'
vectordb_4000 = Chroma(embedding_function=hf, persist_directory=persist_directory)
vectordb_4000.persist()

In [ ]:
persist_directory = '/content/drive/MyDrive/Skripsi_C14210098/5000_DB'
vectordb_5000 = Chroma(embedding_function=hf, persist_directory=persist_directory)
vectordb_5000.persist()

In [ ]:
persist_directory = '/content/drive/MyDrive/Skripsi_C14210098/DB_6000'
vectordb_6000 = Chroma(embedding_function=hf, persist_directory=persist_directory)
vectordb_6000.persist()

In [ ]:
print(f"Length of vectordb_2000: {len(vectordb_2000)}")
print(f"Length of vectordb_3000: {len(vectordb_3000)}")
print(f"Length of vectordb_4000: {len(vectordb_4000)}")
print(f"Length of vectordb_5000: {len(vectordb_5000)}")
print(f"Length of vectordb_6000: {len(vectordb_6000)}")

Length of vectordb_2000: 30350
Length of vectordb_3000: 20289
Length of vectordb_4000: 15295
Length of vectordb_5000: 0
Length of vectordb_6000: 10347


In [ ]:
print(f"Length of vectordb_5000: {len(vectordb_5000)}")

Length of vectordb_5000: 0


##Reranker

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
reranker_model_name= "BAAI/bge-reranker-base"
reranker_tokenizer = AutoTokenizer.from_pretrained(reranker_model_name)
reranker_model = AutoModelForSequenceClassification.from_pretrained(reranker_model_name)

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [ ]:
def rerank_documents(docs, query, top_k):
    """
    Reranks retrieved documents using bge-reranker-base.

    Parameters:
        docs (List[Document]): List of retrieved documents.
        query (str): The original query.
        top_k (int): Number of top-ranked documents to return.

    Returns:
        List[Document]: Reranked list of top_k documents.
    """
    if not docs:
        return []

    # Prepare inputs for reranking
    inputs = [(query, doc.page_content) for doc in docs]

    # Tokenize inputs
    tokenized_inputs = reranker_tokenizer(inputs, return_tensors="pt", padding=True, truncation=True)

    # Get output from reranker model
    with torch.no_grad():
        outputs = reranker_model(**tokenized_inputs)

    # Extract relevance scores from `pooler_output`
    scores = outputs.logits.squeeze().tolist()

    # Combine documents and scores, then sort by score (descending)
    ranked_docs = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)

    # Return top_k documents
    return [doc for doc, _ in ranked_docs[:top_k]]

##Retrieval

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-id")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-id")
def translate(text):
  inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
  outputs = model.generate(**inputs)
  return tokenizer.decode(outputs[0], skip_special_tokens=True)

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/796k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/801k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/291M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
def retriever(query, chroma_db, module_name, alpha=0.5):
    """
    Retrieves relevant documents using an ensemble of BM25 and Chroma retrievers,
    then reranks them using bge-reranker-base.

    Parameters:
        query (str): The user query.
        chroma_db (Chroma): The Chroma database instance.
        module_name (str): The module name to filter documents.
        top_k (int): Number of top-ranked documents to return.

    Returns:
        List[Document]: Final reranked documents.
    """
    # Get all documents from ChromaDB
    all_data = chroma_db.get()

    # Filter documents by module
    filtered_docs = [
        Document(page_content=text, metadata=meta)
        for text, meta in zip(all_data["documents"], all_data["metadatas"])
        if meta.get("module") == module_name
    ]

    if not filtered_docs:
        return []

    # BM25 Retriever (lexical search) with filtered documents
    bm25_retriever = BM25Retriever.from_documents(filtered_docs)
    bm25_retriever.k = 5  # Number of top results to retrieve

    # Chroma Retriever (vector search)
    chroma_retriever = chroma_db.as_retriever(search_kwargs={'k': 5, 'filter': {'module': module_name}})

    # Ensemble Retriever (combining both)
    ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, chroma_retriever], weights=[alpha, 1-alpha])

    # Retrieve documents
    docs = ensemble_retriever.invoke(query)

    # Rerank the retrieved documents using the previous rerank_documents function
    return rerank_documents(docs, query, 2)

In [ ]:
docs = retriever("What is Machine Learning?", vectordb_5000,"Machine Learning")

In [ ]:
print("ENG:")
for doc in docs:
  print("- ", doc.page_content, "\n")

ENG:


In [ ]:
print("ID:")
for doc in docs:
  doc.page_content = translate(doc.page_content)
  print("- ", doc.page_content, "\n")

ID:


## FINAL Pengujian

In [ ]:
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
from sklearn.metrics import ndcg_score
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import time

def evaluate_all_metrics_from_file(csv_path, chroma_db, queries, module_names, alpha=0.5,
                                   relevance_col="Nilai/Ranking (0-3)", query_col="Query", doc_col="Documents content", k=5):
    """
    Evaluasi retrieval dari file CSV: NDCG, cosine similarity, dan waktu retrieval per query.

    Parameters:
        csv_path (str): Path ke file CSV.
        chroma_db: ChromaDB instance.
        module_name (str): Filter modul dokumen.
        alpha (float): Bobot cosine similarity (BM25 = 1 - alpha).
        k (int): Top-k untuk evaluasi NDCG.

    Returns:
        pd.DataFrame: Tabel NDCG per query. Print juga rata-rata semua metrik.
    """
    # Load CSV & inisialisasi
    df = pd.read_csv(csv_path)
    queries = df[query_col].unique().tolist()
    model = SentenceTransformer("all-MiniLM-L6-v2")
    results = []
    total_cosine_values = []
    total_retrieval_time = []

    for query in queries:
        group = df[df[query_col] == query]
        if group.empty:
            continue

        docs = group[doc_col].tolist()
        ground_truth = group[relevance_col].tolist()

        # BM25 lexical score
        tokenized_docs = [doc.lower().split() for doc in docs]
        tokenized_query = query.lower().split()
        bm25 = BM25Okapi(tokenized_docs)
        bm25_scores = bm25.get_scores(tokenized_query)

        # Cosine score (MiniLM)
        query_embedding = model.encode([query])
        doc_embeddings = model.encode(docs)
        cosine_scores = cosine_similarity(query_embedding, doc_embeddings)[0]
        total_cosine_values.extend(cosine_scores)

        # Min-max normalization
        def normalize(scores):
            min_s, max_s = min(scores), max(scores)
            return [(s - min_s) / (max_s - min_s) if max_s != min_s else 0.5 for s in scores]

        bm25_norm = normalize(bm25_scores)
        cosine_norm = normalize(cosine_scores)
        combined = [alpha * c + (1 - alpha) * b for c, b in zip(cosine_norm, bm25_norm)]

        # Hitung NDCG
        ndcg = ndcg_score([ground_truth], [combined], k=k)
        results.append({"Query": query, "NDCG": ndcg})

        # Hitung retrieval time dari retriever milikmu
        for query, module_name in zip(queries, module_names):
          start = time.time()
          _ = retriever(query, chroma_db, module_name, alpha=alpha)
          end = time.time()
          total_retrieval_time.append(end - start)

    # Buat DataFrame hasil evaluasi
    ndcg_df = pd.DataFrame(results)
    avg_ndcg = ndcg_df["NDCG"].mean()
    avg_cosine = sum(total_cosine_values) / len(total_cosine_values) if total_cosine_values else 0.0
    avg_retrieval = sum(total_retrieval_time) / len(total_retrieval_time) if total_retrieval_time else 0.0

    # Cetak summary
    print(f"\n📄 File: {csv_path}")
    print(f"🔢 Average NDCG across all queries: {avg_ndcg:.4f}")
    print(f"📏 Average Cosine Similarity: {avg_cosine:.4f}")
    print(f"⏱️  Average Retrieval Time: {avg_retrieval:.4f} seconds\n")

    return ndcg_df

In [ ]:
queries=["What is the definition of Unsupervised Learning?",
         "What are the main advantages of Particle Swarm Optimization compared to traditional optimization algorithms?",
         "How is AI used in everyday life?",
         "What are some common heuristic search algorithms?",
         "What is an artificial neural network (ANN)?"]
module_names=["Genetic Algorithm and Nature Inspired Algorithms",
              "Particle Swarm Optimization",
              "Artificial Intelligence in Everyday Life",
              "Heuristic Search Algorithms",
              "Neural Network, Deep Learning, and Generative AI"]

In [ ]:
evaluate_all_metrics_from_file("/content/drive/MyDrive/Skripsi_C14210098/Ranking NDCG/Ranking document NDCG - DB_2000.csv", vectordb_2000, queries, module_names)


📄 File: Ranking document NDCG - DB_2000.csv
🔢 Average NDCG across all queries: 0.8835
📏 Average Cosine Similarity: 0.6337
⏱️  Average Retrieval Time: 10.0395 seconds



,Query,NDCG
0,What is the definition of Unsupervised Learning?,1.000000
1,What are the main advantages of Particle Swarm...,0.973012
2,How is AI used in everyday life?,0.675884
3,What are some common heuristic search algorithms?,0.923196
4,What is an artificial neural network (ANN)?,0.845247


In [ ]:
evaluate_all_metrics_from_file("/content/drive/MyDrive/Skripsi_C14210098/Ranking NDCG/Ranking document NDCG - DB_3000.csv", vectordb_2000, queries, module_names)


📄 File: Ranking document NDCG - DB_3000.csv
🔢 Average NDCG across all queries: 0.8518
📏 Average Cosine Similarity: 0.6016
⏱️  Average Retrieval Time: 8.4920 seconds



,Query,NDCG
0,What is the definition of Unsupervised Learning?,0.845643
1,What are the main advantages of Particle Swarm...,0.736930
2,How is AI used in everyday life?,0.989037
3,What are some common heuristic search algorithms?,0.707579
4,What is an artificial neural network (ANN)?,0.979681


In [ ]:
evaluate_all_metrics_from_file("/content/drive/MyDrive/Skripsi_C14210098/Ranking NDCG/Ranking document NDCG - DB_4000.csv", vectordb_2000, queries, module_names)


📄 File: Ranking document NDCG - DB_4000.csv
🔢 Average NDCG across all queries: 0.8591
📏 Average Cosine Similarity: 0.6164
⏱️  Average Retrieval Time: 8.2290 seconds



,Query,NDCG
0,What is the definition of Unsupervised Learning?,0.967577
1,What are the main advantages of Particle Swarm...,0.537933
2,How is AI used in everyday life?,0.891558
3,What are some common heuristic search algorithms?,0.898292
4,What is an artificial neural network (ANN)?,1.000000


In [ ]:
evaluate_all_metrics_from_file("/content/drive/MyDrive/Skripsi_C14210098/Ranking NDCG/Ranking document NDCG - DB_5000.csv", vectordb_2000, queries, module_names)


📄 File: Ranking document NDCG - DB_5000.csv
🔢 Average NDCG across all queries: 0.8975
📏 Average Cosine Similarity: 0.6332
⏱️  Average Retrieval Time: 8.6575 seconds



,Query,NDCG
0,What is the definition of Unsupervised Learning?,0.937778
1,What are the main advantages of Particle Swarm...,0.828274
2,How is AI used in everyday life?,0.848074
3,What are some common heuristic search algorithms?,0.898292
4,What is an artificial neural network (ANN)?,0.975176


In [ ]:
evaluate_all_metrics_from_file("/content/drive/MyDrive/Skripsi_C14210098/Ranking NDCG/Ranking document NDCG - DB_6000.csv", vectordb_2000, queries, module_names)


📄 File: Ranking document NDCG - DB_6000.csv
🔢 Average NDCG across all queries: 0.8612
📏 Average Cosine Similarity: 0.6311
⏱️  Average Retrieval Time: 8.4716 seconds



,Query,NDCG
0,What is the definition of Unsupervised Learning?,0.975528
1,What are the main advantages of Particle Swarm...,0.912547
2,How is AI used in everyday life?,0.676868
3,What are some common heuristic search algorithms?,0.972320
4,What is an artificial neural network (ANN)?,0.768850
